### ### Search Engine with Tools And Agents using ollama

In [170]:
## Arxiv--Research
## Tools creation
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

In [171]:
## Used the inbuilt tool of wikipedia
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [172]:
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
print(arxiv.name)

arxiv


In [173]:
tools=[wiki,arxiv]

In [174]:

!pip install langchain langchain_groq groq

In [175]:
## Custom tools[RAG Tool]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings 

#from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [176]:

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)#RecursiveCharacterTextSplitter is used to split large text into smaller chunks while preserving context as much as possible. It's helpful for embedding, retrieval, and processing large documents.
vectordb=FAISS.from_documents(documents,OllamaEmbeddings(model="mistral"))#FAISS is a vector database that enables fast similarity search. It is commonly used for storing and retrieving high-dimensional embeddings 
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x1297716f0>, search_kwargs={})

In [177]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith-search","Search any information about Langsmith ")

retriever_tool.name

'langsmith-search'

In [178]:
tools=[wiki,arxiv,retriever_tool]

In [179]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/home/Downloads/5-Search Engine/.venv/lib/python3.10/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith-search', description='Search any information about Langsmith ', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x111d740d0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x1297716f0>, 

In [188]:
## Run all this tools with Agents and LLM Models

## Tools, LLM-->AgentExecutor
from langchain_community.chat_models import ChatOllama
from dotenv import load_dotenv
import os
load_dotenv()
llm = ChatOllama(model="llama3")

In [189]:
## Prompt Template
from langchain import hub
prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

/Users/home/Downloads/5-Search Engine/.venv/lib/python3.10/site-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [190]:
## Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [192]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [194]:
print(agent_executor)

verbose=True agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMess

In [196]:
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...
Langsmith!

Langsmith is a small village and civil parish in the West Lindsey district of Lincolnshire, England. It's a quiet rural area with a population of around 300 people.

Here are some interesting facts about Langsmith:

1. **History**: The village has its roots dating back to the Domesday Book (1086), where it was listed as "Langesmes" or "Langesmæ". Over time, the name evolved into Langsmith.
2. **Location**: Langsmith is situated approximately 4 miles (6.4 km) south of Gainsborough and about 25 miles (40 km) west of Lincoln city center.
3. **Landmarks**: The village has a few notable landmarks, including St. Andrew's Church (built in the 13th century), a War Memorial, and some charming rural cottages.
4. **Economy**: Langsmith's economy is primarily based on agriculture, with many residents working in farming or related industries.
5. **Community**: The village has a strong sense of community, with regular events like fetes, fairs, and 

{'input': 'Tell me about Langsmith',
 'output': 'Langsmith!\n\nLangsmith is a small village and civil parish in the West Lindsey district of Lincolnshire, England. It\'s a quiet rural area with a population of around 300 people.\n\nHere are some interesting facts about Langsmith:\n\n1. **History**: The village has its roots dating back to the Domesday Book (1086), where it was listed as "Langesmes" or "Langesmæ". Over time, the name evolved into Langsmith.\n2. **Location**: Langsmith is situated approximately 4 miles (6.4 km) south of Gainsborough and about 25 miles (40 km) west of Lincoln city center.\n3. **Landmarks**: The village has a few notable landmarks, including St. Andrew\'s Church (built in the 13th century), a War Memorial, and some charming rural cottages.\n4. **Economy**: Langsmith\'s economy is primarily based on agriculture, with many residents working in farming or related industries.\n5. **Community**: The village has a strong sense of community, with regular events l

In [ ]:
agent_executor.invoke({"input":"What is machine learning"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks wit
Invoking: `arxiv` with `{'query': 'machine learning'}`


Published: 2019-09-08
Title: Lecture Notes: Optimization for Machine Learning
Authors: Elad Hazan
Summary: Lecture notes on optimization for machine learning, derived from a course at
Princeton University and tutorials given in MLSS, Buenos Aires, as
Invoking: `langsmith-search` with `{'query': 'machine learning'}`


Get started with LangSmith | ü¶úÔ∏èüõ†Ô∏è LangSmith

Get started by adding tracing to your application.
Create dashboards to view key metrics like RPS, error rates and costs.

Evals‚Äã
The quality and development speed of AI applications depends on high-quality evaluat

{'input': 'What is machine learning',
 'output': 'I don\'t think any of the tools I have available are relevant to answering this question. The output from the tools so far has been about machine learning, optimization, and LangSmith, which doesn\'t seem to be related to the original question about machine learning. I\'ll respond with a direct answer:\n\nMachine learning is a subfield of artificial intelligence that involves the use of algorithms and statistical models to enable computers to learn from data, make decisions, and improve their performance over time. The term "machine learning" was coined in the 1950s, but the field has gained significant attention and advancements in recent years due to the availability of large amounts of data and the development of powerful computing resources.\n\nIn essence, machine learning allows computers to learn from experience, recognize patterns, and make predictions or decisions without being explicitly programmed. This is achieved through var

In [197]:
agent_executor.invoke({"input":"What's the paper 1706.03763 about?"})



> Entering new AgentExecutor chain...
Paper 1706.03763 is titled "The Emergence of Complexity in Random Boolean Networks" and was published on the arXiv preprint server in June 2017.

In this paper, the authors study random Boolean networks (RBNs), which are mathematical models that consist of a set of nodes, each representing a binary variable or state. The nodes interact with each other according to certain rules, and the network's behavior is determined by the evolution of these interactions over time.

The authors show that RBNs can exhibit complex behavior, including phase transitions and emergent properties, when the number of nodes in the network becomes sufficiently large. They also investigate how different parameters, such as the connectivity of the nodes or the noise level in the system, affect the emergence of complexity.

The study is significant because it provides insights into the fundamental mechanisms that give rise to complex behavior in many natural and artificial

{'input': "What's the paper 1706.03763 about?",
 'output': 'Paper 1706.03763 is titled "The Emergence of Complexity in Random Boolean Networks" and was published on the arXiv preprint server in June 2017.\n\nIn this paper, the authors study random Boolean networks (RBNs), which are mathematical models that consist of a set of nodes, each representing a binary variable or state. The nodes interact with each other according to certain rules, and the network\'s behavior is determined by the evolution of these interactions over time.\n\nThe authors show that RBNs can exhibit complex behavior, including phase transitions and emergent properties, when the number of nodes in the network becomes sufficiently large. They also investigate how different parameters, such as the connectivity of the nodes or the noise level in the system, affect the emergence of complexity.\n\nThe study is significant because it provides insights into the fundamental mechanisms that give rise to complex behavior in 